In [1]:
import os
import re
import time
import datetime
from tqdm import tqdm 
import pandas as pd
import requests
import ast
from IPython.display import display

In [2]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

c:\pylabs\area-risk-flagging


In [3]:
from utils.configs import BASE_URL, URLS_FRAGMENTS, COLUMN_NAME
from utils.helper_func import build_complete_urls, extract_company_name, combined_df, parse_admin_region, parse_sale_period, find_first_sale_time, sales_start_time, extract_mixed_alphanumeric_ids

In [ ]:
urls = build_complete_urls(BASE_URL, URLS_FRAGMENTS)
print(urls)

In [ ]:
community_raw_df = combined_df(urls, "20250801")

In [ ]:
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\community_data\raw"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
csv_fn = "community_raw_data_20280801.csv"
out_path = os.path.join(output_dir,  csv_fn)
community_raw_df.to_csv(out_path, index=False, encoding='utf-8-sig')

In [4]:
input_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\community_data\raw"
csv_fn = "community_raw_data_20280801.csv"
input_path = os.path.join(input_dir,  csv_fn)

df = pd.read_csv(input_path, encoding='utf-8-sig')

In [ ]:
df.columns

In [5]:
df = df.rename(columns=COLUMN_NAME, inplace=False)
display(df.columns)
display(df.shape)

Index(['縣市', '使用分區', '坐落街道', '申請人', '申請備查期間', '坐落基地', '備查完成日期', '縣市代號',
       '自銷售期間', '完成建物第一次登記日期', '戶數', '編號', '編號列表', '緯度', '建照核發日', '建照執照', '經度',
       '主要建材', '起造人', '社區名稱', '圖片路徑', '主要用途', '代銷售期間', '序號', '子編號', '行政區代號',
       '匯入時間'],
      dtype='object')

(8564, 27)

In [ ]:
# pd.set_option('display.max_columns', None)  # 所有欄位都顯示
# pd.set_option('display.max_rows', 500)      # 最多顯示 100 列
# pd.reset_option('display.max_colwidth', None) # 不截斷欄位內容


In [6]:
proc_df = df.copy()

# 新增欄位 'idlist_cleaned' 存放擷取後的編號清單
proc_df['備查編號清單'] = proc_df[ '編號列表'].apply(extract_mixed_alphanumeric_ids)

# 取出建商名稱，並建立company_name欄位
proc_df['建設公司'] = proc_df.apply(extract_company_name, axis=1)
# 在 proc_df 裡面新增一個「行政區」欄位
proc_df["行政區"] = proc_df["坐落街道"].apply(parse_admin_region)
# proc_df["自售期間"], proc_df["代銷期間"] = zip(*proc_df["銷售期間"].apply(parse_sale_period))

# 依規則從「自售期間」及「代銷期間」欄位提取出7位數字，
# 分別存入新欄位「自售起始時間」與「代銷起始時間」
proc_df["自售起始時間"] = proc_df["自銷售期間"].apply(find_first_sale_time)
proc_df["代銷起始時間"] = proc_df["代銷售期間"].apply(find_first_sale_time)


In [ ]:
# proc_df[['自售起始時間', '代銷起始時間', '自銷售期間', '代銷售期間']].sample(200)
# proc_df[(proc_df['自售起始時間'].isna() & proc_df['代銷起始時間'].isna())][['自銷售期間', '代銷售期間']]

In [7]:
# 建立「銷售起始時間」欄位
proc_df["銷售起始時間"] = proc_df.apply(sales_start_time, axis=1)

In [ ]:
# proc_df[proc_df['銷售起始時間'].isna()]

In [ ]:
proc_df.columns

In [8]:
community_df = proc_df[[ '編號', '縣市', '行政區', '社區名稱', '戶數',  '主要用途', '銷售起始時間', '完成建物第一次登記日期','建照執照',  
                        '自售起始時間', '代銷起始時間', '備查完成日期', '建照核發日', '自銷售期間', '代銷售期間', '經度','緯度', '主要建材','坐落街道','使用分區', '坐落基地', '備查編號清單','匯入時間']]
community_df


,編號,縣市,行政區,社區名稱,戶數,主要用途,銷售起始時間,完成建物第一次登記日期,建照執照,自售起始時間,...,自銷售期間,代銷售期間,經度,緯度,主要建材,坐落街道,使用分區,坐落基地,備查編號清單,匯入時間
0,G1A011007020007,臺北市,大安區,潤泰大安富陽,130,店舖、住宅,1100701,NaN,108建字第0168號,1100701,...,"1100701~自完售止,1100701~售完為止",NaN,121.557978,25.017312,鋼筋混凝土造,大安區臥龍街268號,第三種住宅區,辛亥段四小段482地號,G1A011007020007,20250801
1,G2A011007020003,臺北市,大同區,亞昕華威,130,住商用,1090601,1130709.0,107建字第0001號,1090601,...,"1090601~1091231,事務所",1100501~1111130,121.512361,25.050456,鋼骨RC造,大同區延平北路一段25號旁,商業區,"玉泉段一小段874地號,玉泉段一小段875地號,玉泉段一小段885地號,玉泉段一小段886地...",G2A011007020003,20250801
2,G2A011007050003,臺北市,松山區,敦仰,65,一般零售業甲組(店舖);集合住宅,1101021,NaN,110建字第0139號,None,...,無,110年10月21日起至使照取得6個月止,121.548457,25.050931,鋼骨造,松山區敦化北路56號,敦化南北路特定專用區(B區),"敦化段二小段4地號,敦化段二小段4-1地號,敦化段二小段16地號,敦化段二小段32地號,敦化...","G2A011012080002, G1A011012030003, G2A011009280...",20250801
3,G1A011007050001,臺北市,士林區,原禾賦,27,住商用,1100701,NaN,108建字第0100號,None,...,無,1100701~1101231,121.537836,25.110492,鋼筋混凝土造,士林區德行東路335號,第三種住宅區,"芝蘭段一小段320地號,芝蘭段一小段321地號,芝蘭段一小段325地號,芝蘭段一小段326地...",G1A011007050001,20250801
4,G1A011007050002,臺北市,大安區,富邦藝樹,60,住商用,1100301,NaN,109建字第0243號,None,...,至完銷,1100301~1101031,121.549528,25.044184,鋼骨造,大安區建安里敦化南路一段161巷5弄6號2樓,敦化南北路特定專用區(A)區,"仁愛段六小段244地號,仁愛段六小段263-4地號,仁愛段六小段516地號,仁愛段六小段51...",G1A011007050002,20250801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8559,G1UA11406270002,花蓮縣,新城鄉,恆大秧悅,11,住宅,1140630,NaN,花建執照字第114A0172號,1140630,...,"1140630~自完售止,自備查完成日起至銷售完畢",NaN,121.599691,24.023945,鋼筋混凝土造,新城鄉佳林,特定農業區甲種建築用地,"天佑段650-1地號,天佑段650-2地號",G1UA11406270002,20250801
8560,G2UB11406300001,花蓮縣,鳳林鎮,谷川田‧鳳林,4,住家用,1140630,NaN,114A0049,None,...,無,1140630~1160630,121.451129,23.753199,鋼筋混凝土造,鳳林鎮中正路二段323巷,住宅區,"鳳仁段825-1地號,鳳仁段825-2地號,鳳仁段825-3地號,鳳仁段825-4地號",G2UB11406300001,20250801
8561,G1UA11407030001,花蓮縣,花蓮市,三軒茶屋,16,集合住宅,1140703.0,NaN,花建執造字第114A0141號,None,...,備查完成日起至銷售完畢,NaN,121.603639,23.972617,鋼筋混凝土造,花蓮市南京街,住宅區,福安段569地號,G1UA11407030001,20250801
8562,G2UA11407140001,花蓮縣,花蓮市,美侖同原,13,集合住宅,1140715.0,NaN,花建執照字第113A0370號,None,...,備查完成至售完為止,備查完成至售完為止,121.622177,24.004308,鋼筋混凝土造,花蓮市華民街,住宅區,民意段641-16地號,G2UA11407140001,20250801


In [9]:
# 輸出的資料夾
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\community_data\processed"
os.makedirs(output_dir, exist_ok=True)

In [10]:
csv_fn = "community_data.csv"
out_path = os.path.join(output_dir,  csv_fn)
community_df.to_csv(out_path, index=False, encoding='utf-8-sig')

# pkl_fn   = "預售屋備查.pkl"
# pkl_path = os.path.join(output_dir, pkl_fn)
# proc_df.to_pickle(pkl_path)